In [35]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

In [36]:
df_caso_random = pd.read_csv("instancias/random/indice.txt");

In [37]:
def correr_experimento(cant_threads, metodo, lista_archivos):
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../build/ContarPalabras", cant_threads, metodo, lista_archivos], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise ValueError(F"Hubo un error en la experimentacion para el algoritmo: {metodo} con la instancia {lista_archivos}.")
    
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempos = process.stdout.read().split('\n');
    tiempo_cargas = float(tiempos[0])
    # Leo de STDOUT el tiempo del maximo paralelo si hay
    tiempo_max_paralelo = 0.0
    if (metodo == "true"):
        tiempo_max_paralelo = float(tiempos[1]);
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [tiempo_cargas, tiempo_max_paralelo];

In [38]:
experimentos = [];

### Experimento 1


In [39]:
cantThreads = 10
lista_archivos = ""
with open(F"instancias/random/indice.txt", 'r') as f:
    lista_archivos = f.read();
for threads in range(1, cantThreads):
    experimentos.append(["big_random", str(threads), "true", lista_archivos])

In [33]:
lista_archivos

'../scripts/instancias/random/random-0.txt ../scripts/instancias/random/random-1.txt ../scripts/instancias/random/random-2.txt ../scripts/instancias/random/random-3.txt ../scripts/instancias/random/random-4.txt ../scripts/instancias/random/random-5.txt ../scripts/instancias/random/random-6.txt ../scripts/instancias/random/random-7.txt ../scripts/instancias/random/random-8.txt ../scripts/instancias/random/random-9.txt ../scripts/instancias/random/random-10.txt ../scripts/instancias/random/random-11.txt ../scripts/instancias/random/random-12.txt ../scripts/instancias/random/random-13.txt ../scripts/instancias/random/random-14.txt ../scripts/instancias/random/random-15.txt ../scripts/instancias/random/random-16.txt ../scripts/instancias/random/random-17.txt ../scripts/instancias/random/random-18.txt ../scripts/instancias/random/random-19.txt ../scripts/instancias/random/random-20.txt ../scripts/instancias/random/random-21.txt ../scripts/instancias/random/random-22.txt ../scripts/instancia

In [34]:
columnas = ["dataset", "cant_threads", "metodo", "tiempo_carga", "tiempo_max"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se está ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos_carga = []
    tiempos_max = []
    
    for i in range(0, T):
        tiempos = correr_experimento(experimento[1], experimento[2], experimento[3])
        tiempos_carga.append(tiempos[0]);
        tiempos_max.append(tiempos[1]);
    tiempo_carga = np.median(tiempos_carga);
    tiempo_max = np.median(tiempos_max);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo_carga, tiempo_max]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/exp1.csv", index=False, header=True);

'Experimento: 2/9'

ValueError: Hubo un error en la experimentacion para el algoritmo: true con la instancia ../scripts/instancias/random/random-0.txt ../scripts/instancias/random/random-1.txt ../scripts/instancias/random/random-2.txt ../scripts/instancias/random/random-3.txt ../scripts/instancias/random/random-4.txt ../scripts/instancias/random/random-5.txt ../scripts/instancias/random/random-6.txt ../scripts/instancias/random/random-7.txt ../scripts/instancias/random/random-8.txt ../scripts/instancias/random/random-9.txt ../scripts/instancias/random/random-10.txt ../scripts/instancias/random/random-11.txt ../scripts/instancias/random/random-12.txt ../scripts/instancias/random/random-13.txt ../scripts/instancias/random/random-14.txt ../scripts/instancias/random/random-15.txt ../scripts/instancias/random/random-16.txt ../scripts/instancias/random/random-17.txt ../scripts/instancias/random/random-18.txt ../scripts/instancias/random/random-19.txt ../scripts/instancias/random/random-20.txt ../scripts/instanc